In [1]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import h5py
import datetime

# Root directory of the project
path = '/mnt/sh_flex_storage/malu/venv/CHIPS_MRCNN'
ROOT_DIR = os.path.abspath(path)

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs_chips4")

#train_path = '/mnt/sh_flex_storage/project/xcos_mask/data/trainset.h5'
#train_set = h5py.File(train_path,'r')
#train_images = train_set['input']
#train_masks = train_set['output']
#del train_set
#NUM_TRAIN = train_images.shape[0] # 224*224

test_path = '/mnt/sh_flex_storage/malu/venv/multilayer/datasets/data6_data.h5'
test_set = h5py.File(test_path,'r')
test_images = test_set['input']
test_masks = test_set['output']
del test_set
NUM_TEST = test_images.shape[0]


class ChipsConfig(Config):
    """Configuration for training on the toy shapes dataset.
    Derives from the base Config class and overrides values specific
    to the toy shapes dataset.
    """
    # Give the configuration a recognizable name
    NAME = "chips"

    # CH
    BACKBONE = "resnet50"
    MINI_MASK_SHAPE = (28, 28)
    DETECTION_MAX_INSTANCES = 100
    ROI_POSITIVE_RATIO = 0.16
    USE_MINI_MASK = True

    # TRY CH
    POOL_SIZE = 7
    MASK_POOL_SIZE = 14

    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1  # CH
    IMAGES_PER_GPU = 8

    # Number of classes (including background)
    NUM_CLASSES = 1 + 3  # background + 3 shapes

    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 256
    IMAGE_MAX_DIM = 256

    # Number of training and validation steps per epoch
    #STEPS_PER_EPOCH = NUM_TRAIN // IMAGES_PER_GPU
    VALIDATION_STEPS = 20

    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = (4, 8, 16, 32, 64)  # anchor side in pixels # CH

    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 16  # CH


config = ChipsConfig()
config.display()
class ChipsDataset(utils.Dataset):
    def __init__(self, class_map=None):
        #new_append 
        self.num_null_mask = 0
        self.all_flaw_size = []
        self.subset = ""
        #original
        self._image_ids = []
        self.image_info = []
        # Background is always the first class
        self.class_info = [{"source": "", "id": 0, "name": "BG"}]
        self.source_class_ids = {}

    def load_chips(self, count,subset):
        assert subset in ["train","test"]
        
        self.subset = subset
        
        # Add classes
        self.add_class("chips", 1, "bump")
        self.add_class("chips", 2, "dent")
        self.add_class("chips", 3, "dot")

        # Add images
        # Images are generated on the fly in load_image().
        for i in range(count):
            chips = self.mask_prepare(i)
            self.add_image("chips", image_id=i, path=None,chips=chips)
            
    
    def mask_prepare(self,i,return_mask=False):
        if self.subset == "train":
            mask_full = train_masks[i,:].reshape((4,224,224))
        else :
            mask_full = test_masks[i,:].reshape((4,224,224))
        
        bump,dent,dot = mask_full[1,:,:],mask_full[2,:,:],mask_full[3,:,:]   
        flaw_size = [np.sum(bump),np.sum(dent),np.sum(dot)]
        names = ["bump","dent","dot"]

        if not np.sum(flaw_size):
            self.num_null_mask += 1
            #print("==== no flaw for image %d ===="%i)

        if not return_mask:
            chips= []
            for j in range(3):
                if flaw_size[j] >0:
                    chips.append(names[j])
                    self.all_flaw_size.append(flaw_size[j])
            return chips
        else:
            masks = []
            for j in range(3):
                if flaw_size[j] >0:
                    masks.append(mask_full[j+1,:,:])
            if not np.array(masks).shape[0]:
                return np.array(masks)
            else:
                masks = np.array(masks)
                masks = masks.reshape((-1,224,224))
                masks_reshape = np.zeros((224,224,masks.shape[0]))
                #assert masks.shape[1:] in [(224,224)]
                for i in range(224):
                    for j in range(224):
                        masks_reshape[i,j,:] = masks[:,i,j]  
                return masks_reshape
                      
        
    def load_image(self,image_id):
        info = self.image_info[image_id]
        
        #print("check load_image image_id: ",image_id)
        
        if self.subset == "train":
            image_full = train_images[image_id,:].reshape((6,224,224))
        else:
            image_full = test_images[image_id,:].reshape((6,224,224))
            
        image_sel = np.array([image_full[0,:,:],image_full[1,:,:],image_full[2,:,:]])
        image_sel_reshape = np.zeros((224,224,3))
        for i in range(224):
            for j in range(224):
                image_sel_reshape[i,j,:] = image_sel[:,i,j]  
        image = image_sel_reshape
        return image

    def image_reference(self, image_id):
        info = self.image_info[image_id]
        if info["source"] == "chips":
            return info["chips"]
        else:
            super(self.__class__).image_reference(self, image_id)

    def load_mask(self, image_id):
        """Generate instance masks for shapes of the given image ID.
        """
        info = self.image_info[image_id]
        chips = info['chips']
        mask = self.mask_prepare(image_id,True)
        
        class_ids = np.array([self.class_names.index(s) for s in chips])
        return mask.astype(np.bool), class_ids.astype(np.int32)

    def info_display(self,count):
        #flaw_size info
        max_flaw_size = np.max(self.all_flaw_size)
        min_flaw_size = np.min(self.all_flaw_size)
        mean_flaw_size = np.mean(self.all_flaw_size)
        
        #flaw_kind_info
        bump_num,dent_num,dot_num = 0,0,0
        for i in range(count):
            chips = self.image_info[i]['chips']
            if "bump" in chips:
                bump_num +=1
            if "dent" in chips:
                dent_num +=1
            if "dot" in chips:
                dot_num += 1    
                
        #display all info
        print("**** info of %s dataset : ***  "%self.subset)
        print("  ratio of null mask images:%d %%"%(self.num_null_mask/count*100))
        print("  max flaw size:",max_flaw_size)
        print("  min flaw size:",min_flaw_size)
        print("  mean flaw size:",mean_flaw_size)
        print("  bump ratio:%d %%"%(bump_num/count*100))
        print("  dent ratio:%d %%"%(dent_num/count*100))
        print("  dot ratio:%d %%"%(dot_num/count*100))
    
dataset_test = ChipsDataset()
dataset_test.load_chips(NUM_TEST,"test")
dataset_test.prepare()
dataset_test.info_display(NUM_TEST)

# dataset_train = ChipsDataset()
# dataset_train.load_chips(NUM_TRAIN,"train")
# dataset_train.prepare()
# dataset_train.info_display(NUM_TRAIN)

modified at 090417


Using TensorFlow backend.



Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     8
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 8
IMAGE_MAX_DIM                  256
IMAGE_META_SIZE                16
IMAGE_MIN_DIM                  256
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [256 256   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'mrcnn_mask_loss': 1.0, 'rpn_bbox_loss': 1.0, 'rpn_class_loss': 2.0, 'mrcnn_class_loss': 2.0, 'mrcnn_bbox_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES 

In [5]:
import warnings
warnings.filterwarnings('ignore')

class InferenceConfig(ChipsConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    
inference_config = InferenceConfig()

# MODEL_DIR = os.path.join(ROOT_DIR, "logs_chips3")
model_path = MODEL_DIR+'/chips20181016T1457/mask_rcnn_chips_0012.h5' 
#model_path = 'mask_rcnn_chips_3_0905_null_train.h5'
# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir= MODEL_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
#model_path = os.path.join(MODEL_DIR, "mask_rcnn_chips.h5")
#model_path = model.find_last()

# Load trained weights
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

import pandas as pd
import time 

def model_eval(dataset,name):
    start = time.time()
    if name == 'train':
        image_ids = dataset.image_ids
    else:
        image_ids = dataset.image_ids
        
    gt_class = []
    pre_class = []
    ap = []
    score = []
    
    for image_id in image_ids:
        print('working on %d'%image_id)
        # Load image and ground truth data
        image, image_meta, gt_class_id, gt_bbox, gt_mask =\
            modellib.load_image_gt(dataset, inference_config,
                                   image_id, use_mini_mask=False)
        # molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
        # Run object detection
        results = model.detect([image], verbose=0)
        r = results[0]

        # Compute AP
        if len(r['class_ids'])>0 and len(gt_class_id)>0 :
            AP, precisions, recalls, overlaps =utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                                 r["rois"], r["class_ids"], r["scores"], r['masks'],iou_threshold=0.5)
            #print(AP)
            ap.append(AP)
        else:
            ap.append(0)

        if not len(r['class_ids']) :
            pre_class.append(0)
            score.append(0)
        else:
            pre_class.append(r['class_ids'][0])
            score.append(r['scores'][0])

        if not len(gt_class_id):
            gt_class.append(0)
        else:
            gt_class.append(gt_class_id[0])
    
   
    # predict summary
    result = pd.DataFrame()
    result['image_id'] = image_ids
    result['gt_class'] = gt_class
    result['pre_class'] = pre_class
    result['ap'] = ap
    result['score']=score
    
    return result 

def info_display(result):
    
    image_ids = result['image_id']
    gt_class = result['gt_class']
    pre_class = result['pre_class']
    ap = result['ap']
    
    #result display
    diff = np.array(gt_class)-np.array(pre_class)
    diff = diff.tolist()
    precision = diff.count(0)/len(diff)
    total_false = len(diff)-diff.count(0)
    
    negative = len(result[result['gt_class']==0])
    positive = len(result)-negative
    
    false_positive = len(result[result['gt_class']==0][result['pre_class']!=0])
    false_negative = len(result[result['gt_class']!=0][result['pre_class']==0])
    
    #print('evaluation on %s'%name)
    print('mAP = ',np.mean(ap))
    print('precision =  %d%%'%(precision*100))
    print('false_positive/total = %d%%'%(false_positive/len(image_ids)*100))
    print('false_negative/toal = %d%%'%(false_negative/len(image_ids)*100))
  
    print('false_positive/negative = %d%%'%((false_positive/negative)*100))
    print('false_negative/positive = %d%%'%((false_negative/positive)*100))
    
test_result = model_eval(dataset_test,'test')


Loading weights from  /mnt/sh_flex_storage/malu/venv/CHIPS_MRCNN/logs_chips4/chips20181016T1457/mask_rcnn_chips_0012.h5
load weight from /mnt/sh_flex_storage/malu/venv/CHIPS_MRCNN/logs_chips4/chips20181016T1457/mask_rcnn_chips_0012.h5 by name
Re-starting from epoch 12
working on 0
working on 1
working on 2
working on 3
working on 4
working on 5
working on 6
working on 7
working on 8
working on 9
working on 10
working on 11
working on 12
working on 13
working on 14
working on 15
working on 16
working on 17
working on 18
working on 19
working on 20
working on 21
working on 22
working on 23
working on 24
working on 25
working on 26
working on 27
working on 28
working on 29
working on 30
working on 31
working on 32
working on 33
working on 34
working on 35
working on 36
working on 37
working on 38
working on 39
working on 40
working on 41
working on 42
working on 43
working on 44
working on 45
working on 46
working on 47
working on 48
working on 49
working on 50
working on 51
working on 52

working on 537
working on 538
working on 539
working on 540
working on 541
working on 542
working on 543
working on 544
working on 545
working on 546
working on 547
working on 548
working on 549
working on 550
working on 551
working on 552
working on 553
working on 554
working on 555
working on 556
working on 557
working on 558
working on 559
working on 560
working on 561
working on 562
working on 563
working on 564
working on 565
working on 566
working on 567
working on 568
working on 569
working on 570
working on 571
working on 572
working on 573
working on 574
working on 575
working on 576
working on 577
working on 578
working on 579
working on 580
working on 581
working on 582
working on 583
working on 584
working on 585
working on 586
working on 587
working on 588
working on 589
working on 590
working on 591
working on 592
working on 593
working on 594
working on 595
working on 596
working on 597
working on 598
working on 599
working on 600
working on 601
working on 602
working on

working on 1079
working on 1080
working on 1081
working on 1082
working on 1083
working on 1084
working on 1085
working on 1086
working on 1087
working on 1088
working on 1089
working on 1090
working on 1091
working on 1092
working on 1093
working on 1094
working on 1095
working on 1096
working on 1097
working on 1098
working on 1099
working on 1100
working on 1101
working on 1102
working on 1103
working on 1104
working on 1105
working on 1106
working on 1107
working on 1108
working on 1109
working on 1110
working on 1111
working on 1112
working on 1113
working on 1114
working on 1115
working on 1116
working on 1117
working on 1118
working on 1119
working on 1120
working on 1121
working on 1122
working on 1123
working on 1124
working on 1125
working on 1126
working on 1127
working on 1128
working on 1129
working on 1130
working on 1131
working on 1132
working on 1133
working on 1134
working on 1135
working on 1136
working on 1137
working on 1138
working on 1139
working on 1140
working 

working on 1593
working on 1594
working on 1595
working on 1596
working on 1597
working on 1598
working on 1599
working on 1600
working on 1601
working on 1602
working on 1603
working on 1604
working on 1605
working on 1606
working on 1607
working on 1608
working on 1609
working on 1610
working on 1611
working on 1612
working on 1613
working on 1614
working on 1615
working on 1616
working on 1617
working on 1618
working on 1619
working on 1620
working on 1621
working on 1622
working on 1623
working on 1624
working on 1625
working on 1626
working on 1627
working on 1628
working on 1629
working on 1630
working on 1631
working on 1632
working on 1633
working on 1634
working on 1635
working on 1636
working on 1637
working on 1638
working on 1639
working on 1640
working on 1641
working on 1642
working on 1643
working on 1644
working on 1645
working on 1646
working on 1647
working on 1648
working on 1649
working on 1650
working on 1651
working on 1652
working on 1653
working on 1654
working 

working on 2107
working on 2108
working on 2109
working on 2110
working on 2111
working on 2112
working on 2113
working on 2114
working on 2115
working on 2116
working on 2117
working on 2118
working on 2119
working on 2120
working on 2121
working on 2122
working on 2123
working on 2124
working on 2125
working on 2126
working on 2127
working on 2128
working on 2129
working on 2130
working on 2131
working on 2132
working on 2133
working on 2134
working on 2135
working on 2136
working on 2137
working on 2138
working on 2139
working on 2140
working on 2141
working on 2142
working on 2143
working on 2144
working on 2145
working on 2146
working on 2147
working on 2148
working on 2149
working on 2150
working on 2151
working on 2152
working on 2153
working on 2154
working on 2155
working on 2156
working on 2157
working on 2158
working on 2159
working on 2160
working on 2161
working on 2162
working on 2163
working on 2164
working on 2165
working on 2166
working on 2167
working on 2168
working 

working on 2621
working on 2622
working on 2623
working on 2624
working on 2625
working on 2626
working on 2627
working on 2628
working on 2629
working on 2630
working on 2631
working on 2632
working on 2633
working on 2634
working on 2635
working on 2636
working on 2637
working on 2638
working on 2639
working on 2640
working on 2641
working on 2642
working on 2643
working on 2644
working on 2645
working on 2646
working on 2647
working on 2648
working on 2649
working on 2650
working on 2651
working on 2652
working on 2653
working on 2654
working on 2655
working on 2656
working on 2657
working on 2658
working on 2659
working on 2660
working on 2661
working on 2662
working on 2663
working on 2664
working on 2665
working on 2666
working on 2667
working on 2668
working on 2669
working on 2670
working on 2671
working on 2672
working on 2673
working on 2674
working on 2675
working on 2676
working on 2677
working on 2678
working on 2679
working on 2680
working on 2681
working on 2682
working 

working on 3135
working on 3136
working on 3137
working on 3138
working on 3139
working on 3140
working on 3141
working on 3142
working on 3143
working on 3144
working on 3145
working on 3146
working on 3147
working on 3148
working on 3149
working on 3150
working on 3151
working on 3152
working on 3153
working on 3154
working on 3155
working on 3156
working on 3157
working on 3158
working on 3159
working on 3160
working on 3161
working on 3162
working on 3163
working on 3164
working on 3165
working on 3166
working on 3167
working on 3168
working on 3169
working on 3170
working on 3171
working on 3172
working on 3173
working on 3174
working on 3175
working on 3176
working on 3177
working on 3178
working on 3179
working on 3180
working on 3181
working on 3182
working on 3183
working on 3184
working on 3185
working on 3186
working on 3187
working on 3188
working on 3189
working on 3190
working on 3191
working on 3192
working on 3193
working on 3194
working on 3195
working on 3196
working 

working on 3649
working on 3650
working on 3651
working on 3652
working on 3653
working on 3654
working on 3655
working on 3656
working on 3657
working on 3658
working on 3659
working on 3660
working on 3661
working on 3662
working on 3663
working on 3664
working on 3665
working on 3666
working on 3667
working on 3668
working on 3669
working on 3670
working on 3671
working on 3672
working on 3673
working on 3674
working on 3675
working on 3676
working on 3677
working on 3678
working on 3679
working on 3680
working on 3681
working on 3682
working on 3683
working on 3684
working on 3685
working on 3686
working on 3687
working on 3688
working on 3689
working on 3690
working on 3691
working on 3692
working on 3693
working on 3694
working on 3695
working on 3696
working on 3697
working on 3698
working on 3699
working on 3700
working on 3701
working on 3702
working on 3703
working on 3704
working on 3705
working on 3706
working on 3707
working on 3708
working on 3709
working on 3710
working 

In [7]:
info_display(test_result)

mAP =  0.626642845408247
precision =  80%
false_positive/total = 4%
false_negative/toal = 5%
false_positive/negative = 25%
false_negative/positive = 6%


In [12]:
from sklearn.metrics import confusion_matrix as cm
def error_display(result, num=1):
    y_true = result['gt_class']
    y_pred = result['pre_class']
    cmatrix = cm(y_true, y_pred)

    num_finechips = sum(cmatrix[0])
    num_flawchips = len(result) - num_finechips
    num_pre_finechips = sum(cmatrix[:, 0])
    num_pre_flawchips = len(result) - num_pre_finechips

    print('confusion matrix:\n')
    print(cmatrix)

    pres = []
    recs = []
    for i in range(4):
        precison = cmatrix[i, i] / (sum(cmatrix[i]) +1)* 100
        recall = cmatrix[i, i] / (sum(cmatrix[:, i])+1) * 100
        pres.append(precison)
        recs.append(recall)
        print('precision and recall on class%d ： %d%%   %d%% \n' % (i, precison, recall))

    print('total validation samples num ：  %d' % len(result))
    print('mean presicion ： %d%%' % (np.mean(pres)))
    print('mean recall ： %d%%' % (np.mean(recs)))
    print('mean ap ：  %d%% ' % (np.mean(result['ap']) * 100))
    return


In [13]:
error_display(test_result)

confusion matrix:

[[ 537  103   71   12]
 [  97  754   88    7]
 [  86  112 1161   80]
 [  25   29   72  824]]
precision and recall on class0 ： 74%   71% 

precision and recall on class1 ： 79%   75% 

precision and recall on class2 ： 80%   83% 

precision and recall on class3 ： 86%   89% 

total validation samples num ：  4058
mean presicion ： 80%
mean recall ： 79%
mean ap ：  62% 


### model eval at epoch 5 (test_chips3_15.csv)
- mAP =  0.7668939776617837
- precision =  96%
- false_positive/total = 1%
- false_negative/toal = 1%
- false_positive/negative = 6%
- false_negative/positive = 1%

In [9]:
test_result.to_csv('test_chips3_data4_1015.csv')